# Image Classification Problem - Classify Cat vs Dog

## Data Pre-processing

In [1]:
#code from keras documentation - https://keras.io/preprocessing/image/
import os
from keras.preprocessing.image import ImageDataGenerator
#Image augmentation - rotate, shift, flip, etc.. copies of image to expand size of training set
#rescale in a range from zero to 1

train_data_generator = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_data_generator.flow_from_directory('dataset/training_set',
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory('dataset/test_set',
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

Using TensorFlow backend.


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Convolutional Neural Network implementation

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

#Initializing the Convolutional Neural Network

def build_cnn():
    #Step 1 - Convolution
    classifier = Sequential()

    #add feature detector number of rows and columns as kernel size parameter
    #image 64 x 64 with 3 color channels
    #relu activation function to remove linearity
    cnn_input_layer = Conv2D(filters = 32, kernel_size = (3, 3), input_shape = (128, 128, 3), activation = 'relu')
    classifier.add(cnn_input_layer)
    #Step 2 - Max Pooling

    #reduce size of feature maps using max pooling
    max_pooling_layer = MaxPooling2D(pool_size = (2, 2))
    classifier.add(max_pooling_layer)
    
    #Create Deep Fully Connected Layers (hidden layers) - 2nd Layer
    fcl = Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu')
    classifier.add(fcl)
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    
    fcl2 = Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu')
    classifier.add(fcl2)
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    #Step 3 - Flattening
    flattening_layer = Flatten()
    classifier.add(flattening_layer)

    #Step 4 - Full Connection
    ann_fully_connected_layer = Dense(units = 128, activation = 'relu') #same as ANN hidden layer
    classifier.add(Dropout(rate = 0.1))
    classifier.add(ann_fully_connected_layer)

    ann_output_layer = Dense(units = 1, activation = 'sigmoid')
    classifier.add(ann_output_layer)
    return classifier

## Fitting CNN

In [3]:
classifier = build_cnn()
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [4]:
batch_size = 32
classifier.fit_generator(generator = train_generator, steps_per_epoch = 8000/batch_size, epochs = 90, validation_data = test_generator, workers = 16, verbose=1)

Epoch 1/90
250/250 [==============================] - 86s 344ms/step - loss: 0.6891 - accuracy: 0.5290 - val_loss: 0.7270 - val_accuracy: 0.5900
Epoch 2/90
250/250 [==============================] - 85s 341ms/step - loss: 0.6440 - accuracy: 0.6314 - val_loss: 0.5730 - val_accuracy: 0.6970
Epoch 3/90
250/250 [==============================] - 146s 583ms/step - loss: 0.5887 - accuracy: 0.6821 - val_loss: 0.6062 - val_accuracy: 0.7165
Epoch 4/90
250/250 [==============================] - 82s 327ms/step - loss: 0.5460 - accuracy: 0.7226 - val_loss: 0.7758 - val_accuracy: 0.7180
Epoch 5/90
250/250 [==============================] - 78s 313ms/step - loss: 0.5167 - accuracy: 0.7401 - val_loss: 0.2891 - val_accuracy: 0.7650
Epoch 6/90
250/250 [==============================] - 80s 319ms/step - loss: 0.4969 - accuracy: 0.7580 - val_loss: 0.3818 - val_accuracy: 0.7725
Epoch 7/90
250/250 [==============================] - 86s 344ms/step - loss: 0.4632 - accuracy: 0.7822 - val_loss: 0.4082 - val_a

250/250 [==============================] - 94s 374ms/step - loss: 0.0916 - accuracy: 0.9678 - val_loss: 0.2870 - val_accuracy: 0.8505
Epoch 58/90
250/250 [==============================] - 88s 353ms/step - loss: 0.0861 - accuracy: 0.9691 - val_loss: 0.5855 - val_accuracy: 0.8575
Epoch 59/90
250/250 [==============================] - 92s 370ms/step - loss: 0.0750 - accuracy: 0.9715 - val_loss: 0.1944 - val_accuracy: 0.8460
Epoch 60/90
250/250 [==============================] - 105s 421ms/step - loss: 0.0721 - accuracy: 0.9731 - val_loss: 0.5890 - val_accuracy: 0.8435
Epoch 61/90
250/250 [==============================] - 104s 416ms/step - loss: 0.0716 - accuracy: 0.9741 - val_loss: 0.0266 - val_accuracy: 0.8455
Epoch 62/90
250/250 [==============================] - 96s 385ms/step - loss: 0.0855 - accuracy: 0.9695 - val_loss: 0.9776 - val_accuracy: 0.8345
Epoch 63/90
250/250 [==============================] - 96s 386ms/step - loss: 0.0755 - accuracy: 0.9744 - val_loss: 1.1147 - val_accur

## Making new prediction

In [6]:
import numpy as np
from keras.preprocessing import image

#Load Image
dog_or_cat_test1 = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(128, 128))
dog_or_cat_test1 = image.img_to_array(dog_or_cat_test1)

#add dimension for neural network prediction
dog_or_cat_test1 = np.expand_dims(dog_or_cat_test1, axis = 0) #added to beginning of array
#make prediction
binary_result = classifier.predict(dog_or_cat_test1)
binary_result

array([[1.]], dtype=float32)

In [7]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}